In [209]:
import warnings
warnings.filterwarnings('ignore')

In [210]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [211]:
df = pd.read_csv('Mental_Health_FAQ.csv')
df = df.drop('Question_ID', axis=1)
df.head()

,Questions,Answers
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [212]:
questions = [q for q in df['Questions'].values]
answers = [a for a in df['Answers'].values]

In [213]:
question = "Are there any differences between depression and sadness?"
questions.append(question)

In [214]:
nlp = spacy.load("en_core_web_sm")

In [215]:
lemmatized_q = []

for q in questions:
  doc = nlp(q)
  lemmatized_q.append(" ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct]))

In [216]:
lemmatized_q[:5]

['mean mental illness',
 'mental illness affect',
 'cause mental illness',
 'warning sign mental illness',
 'people mental illness recover']

In [217]:
cv = CountVectorizer()
cv_q = cv.fit_transform(lemmatized_q)

In [218]:
tf = TfidfVectorizer()
tf_idf_q = tf.fit_transform(lemmatized_q)

In [219]:
cv_df = pd.DataFrame(cv_q.toarray(), columns=cv.get_feature_names_out())
cv_df.head()

,19,25,addiction,adhd,adult,advance,affect,age,alcohol,allow,...,unwell,use,vaping,warning,well,will,work,worried,young,youth
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [220]:
tf_df = pd.DataFrame(tf_idf_q.toarray(), columns=tf.get_feature_names_out())
tf_df.head()

,19,25,addiction,adhd,adult,advance,affect,age,alcohol,allow,...,unwell,use,vaping,warning,well,will,work,worried,young,youth
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.750128,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.600065,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [221]:
q_cv = cv_df.to_numpy()
q_tf = tf_df.to_numpy()

In [222]:
from scipy.spatial.distance import cosine, jaccard
from scipy.stats import pearsonr
from sklearn.metrics.cluster import rand_score

COS

In [223]:
best_cv_cos = []
res_cv_cos = ''

for i in range(len(questions) - 1):
  best_cv_cos.append(1 - cosine(q_cv[i, :], q_cv[-1, :]))

res_cv_cos = max(best_cv_cos)

print("cos_cv:", question)
print(answers[best_cv_cos.index(res_cv_cos)])

cos_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthless or

In [224]:
best_tf_cos = []
res_tf_cos = ''

for i in range(len(questions) - 1):
  best_tf_cos.append(1 - cosine(q_tf[i, :], q_tf[-1, :]))

res_tf_cos = max(best_tf_cos)

print("cos_tf_idf:", question)
print(answers[best_tf_cos.index(res_tf_cos)])

cos_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthles

Pearson

In [225]:
best_cv_p = []
res_cv_p = ''

for i in range(len(questions) - 1):
  best_cv_p.append(pearsonr(q_cv[i, :], q_cv[-1, :]))

res_cv_p = max(best_cv_p)

print("pearson_cv:", question)
print(answers[best_cv_p.index(res_cv_p)])

pearson_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthles

In [226]:
best_tf_p = []
res_tf_p = ''

for i in range(len(questions) - 1):
  best_tf_p.append(pearsonr(q_tf[i, :], q_tf[-1, :]))

res_tf_p = max(best_tf_p)

print("pearson_tf_idf:", question)
print(answers[best_tf_p.index(res_tf_p)])

pearson_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel wort

Jaccard

In [227]:
best_cv_j = []
res_cv_j = ''

for i in range(len(questions) - 1):
  best_cv_j.append(1 - jaccard(q_cv[i, :], q_cv[-1, :]))

res_cv_j = max(best_cv_j)

print("jaccard_cv:", question)
print(answers[best_cv_j.index(res_cv_j)])

jaccard_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthles

In [228]:
best_tf_j = []
res_tf_j = ''

for i in range(len(questions) - 1):
  best_tf_j.append(1 - jaccard(q_tf[i, :], q_tf[-1, :]))

res_tf_j = max(best_tf_j)

print("jaccard_tf_idf:", question)
print(answers[best_tf_j.index(res_tf_j)])

jaccard_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel wort

Rand_score

In [229]:
best_cv_r = []
res_cv_r = ''

for i in range(len(questions) - 1):
  best_cv_r.append(rand_score(q_cv[i, :], q_cv[-1, :]))

res_cv_r = max(best_cv_r)

print("rand_score_cv:", question)
print(answers[best_cv_r.index(res_cv_r)])

rand_score_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worth

In [230]:
best_tf_r = []
res_tf_r = ''

for i in range(len(questions) - 1):
  best_tf_r.append(rand_score(q_tf[i, :], q_tf[-1, :]))

res_tf_r = max(best_tf_r)

print("rand_score_tf_idf:", question)
print(answers[best_tf_r.index(res_tf_r)])

rand_score_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel w